In [6]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import datetime
import time

In [7]:
dfOscar = pd.read_csv("the_oscar_award.csv")
# an exmaple year
year = 2018
dfOscar1Yr = dfOscar[dfOscar['year_film']==year]
# dfOscar1Yr[['film','winner']].groupby('film').any().shape
dfOscar1Yr

,year_film,year_ceremony,ceremony,category,name,film,winner
10142,2018,2019,91,ACTOR IN A LEADING ROLE,Christian Bale,Vice,False
10143,2018,2019,91,ACTOR IN A LEADING ROLE,Bradley Cooper,A Star Is Born,False
10144,2018,2019,91,ACTOR IN A LEADING ROLE,Willem Dafoe,At Eternity's Gate,False
10145,2018,2019,91,ACTOR IN A LEADING ROLE,Rami Malek,Bohemian Rhapsody,True
10146,2018,2019,91,ACTOR IN A LEADING ROLE,Viggo Mortensen,Green Book,False
...,...,...,...,...,...,...,...
10262,2018,2019,91,WRITING (Original Screenplay),Written by Adam McKay,Vice,False
10263,2018,2019,91,HONORARY AWARD,Marvin Levy,NaN,True
10264,2018,2019,91,HONORARY AWARD,Lalo Schifrin,NaN,True
10265,2018,2019,91,HONORARY AWARD,Cicely Tyson,NaN,True


In [8]:
dfMovies = pd.read_csv("movie_lense/movies.csv")
reWithYr = r'.*\ *\([0-9]+.*\)'
# re1Yr = r'.*\ \(%d\)'%(year) 
dfMovies['film'] = dfMovies['title'].str.replace(r"\ \([0-9]+\)", "")

dfMovies['year_film'] = dfMovies[dfMovies['title'].str.match(reWithYr) == True]['title'].str.replace(r'.*\ *\(',"")
dfMovies['year_film'] = dfMovies['year_film'].str.slice(start=0, stop=4, step=1)
dfMovies.at[35140, 'year_film'] = '1957' # 1 special case
dfMovies['year_film'] = dfMovies['year_film'].astype(float)

In [291]:
dfOscarMovies = dfOscar.merge(dfMovies, left_on='film', right_on='film')
# year_film difference (between differernt dataset) should not exceed 1 year
dfOscarMovies = dfOscarMovies[abs(dfOscarMovies['year_film_x'] - dfOscarMovies['year_film_y']) <= 1]
dfOscarMovies

,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y
2,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,25768,Sadie Thompson (1928),Drama,1928.0
7,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,True,87290,Two Arabian Knights (1927),Adventure|Comedy|Romance,1927.0
8,1927,1928,1,DIRECTING (Comedy Picture),Ted Wilde,Speedy,False,32551,Speedy (1928),Action|Comedy,1928.0
9,1927,1928,1,ENGINEERING EFFECTS,Roy Pomeroy,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
10,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0


In [292]:
ambiCheck = dfOscarMovies[['year_film_y', 'title']].groupby('title').apply(lambda x: len(list(set(x['year_film_y'])))) # no ambiguity
ambiCheck2 = dfOscarMovies[['year_film_y', 'title']].groupby('title').apply(lambda x: list(set(x['year_film_y'])))
ambiCheck2[ambiCheck[ambiCheck != 1].index]
# dfOscarMovies[dfOscarMovies['film']=='Alice in Wonderland']  # same-name, different-year movies

Series([], dtype: object)

In [12]:
dfRatings = pd.read_csv('movie_lense/ratings.csv')
# d = datetime.datetime(1970, 1, 1, 0)
# print(time.mktime(d.timetuple()))
# dfRatings[dfRatings['timestamp']< 18000] # no rating before 1970-1-1 (Unix Epoch)

In [319]:
def get_preRatings(df, rating_df, mode='mode'):
    # collect the ratings before Feb in this year
    d = datetime.datetime(df['year_ceremony'], 2, 1, 0)
    timestamp1 = time.mktime(d.timetuple()) # local time
    if mode == 'mode':
        output = rating_df[(rating_df['timestamp']<timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mode().mean()
    else:
        output = rating_df[(rating_df['timestamp']<timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mean()
    print(df['movieId'])
    return output # return mode/mean
preRatingsMode = lambda x: get_preRatings(x, dfRatings)
preRatingsMean = lambda x: get_preRatings(x, dfRatings, 'mean')
dfOM_1970 = dfOscarMovies[dfOscarMovies['year_film_x']>=1970] # for get the preRatings, use the dataset after 1970
dfOM_1970['preRatings_mode'] = dfOM_1970.apply(preRatingsMode, axis=1) 
dfOM_1970['preRatings_mean'] = dfOM_1970.apply(preRatingsMean, axis=1) 
dfOM_1970.to_csv('oscar_movies_preRat.csv')
dfOM_1970

838
26958
838
26958
261
261
261
5777
5777
5777
192385
192385
192385
192385
192385
192385
192385
192385
45880
4132
132532
2640
2640
2640
2640
26491
2779
2779
2779
2779
2779
2779
2779
2779
2779
1199
1199
6331
1224
1224
1224
3723
3723
1411
1411
1411
1411
5838
2429
1277
1277
1277
1277
1277
130073
97057
74789
74789
74789
4308
4308
4308
4308
4308
4308
4308
4308
1345
1345
8025
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
7
7
2476
41
41
1688
1688
170611
2112
187273
137857
82459
82459
82459
82459
82459
82459
82459
82459
82459
82459
74266
74266
74266
3201
3201
3201
3201
4296
4296
4296
4296
4296
4296
4296
1272
1272
1272
1272
1272
1272
1272
1272
1272
1272
8757
8757
8757
3037
7111
7111
7111
7111
6247
6247
6247
6247
78742
2520
2520
2520
2520
2520
2520
2520
2520
2520
2520
6849
6849
6849
6849
3066
3066
3066
3066
3066
8272
8272
74547
74547
74547
4282
7836
7836
7836
141454
52528
118

1704
1704
1704
1633
1747
1747
1784
1784
1784
1784
1784
1784
1784
1729
1693
1693
1693
1693
1673
1673
1673
1733
1617
1617
1617
1617
1617
1617
1617
1617
1617
1614
1653
1730
1730
1730
1730
1780
4217
6667
4459
1608
1608
1569
1566
1552
1552
95654
1573
1676
1466
1701
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2333
2333
2333
2439
2439
2329
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2336
2336
2336
2336
2336
2336
2336
2272
2442
2442
1810
1810
2390
2321
2321
2321
2297
2297
2314
2337
2538
101062
3609
1912
1912
1907
2431
1917
1917
1917
1917
1881
2384
190219
4454
1883
3129
3129
2858
2858
2858
2858
2858
2858
2858
2858
3160
3160
3160
3118
2996
2996
2908
2908
3186
2997
2997
2997
3155
3155
3081
3081
3081
3163
3163
3163
3163
3185
3181
2677
7786
2824
3989
2683
3156
2587
3179
3114
2687
186343
2959
3157
2599
4021
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
4022
4022
4017
4017
3993
3993
3993
4036
4036
4034
4034
4034
4034
4034
3408
3408
3408
3408
3408
4014
401

166705
166705
166705
161582
161582
161582
161582
165551
165551
165551
165551
165551
165551
161966
163959
178861
166486
166486
166486
155659
155659
166643
166643
166643
162578
162578
166461
166461
160848
152081
164179
164179
164179
164179
164179
164179
164179
164179
143367
164981
135143
135143
164019
167064
160874
160289
164917
169784
169788
169810
167870
156944
151113
151113
164949
165709
158972
135569
135536
166635
166635
165141
152782
143859
168750
165413
168752
168748
160718
169532
167036
169530
169534
162598
162598
160980
166528
166528
122922
133771
166788
168492
168492
168492
168492
181315
181315
181315
181315
181315
181315
168250
168250
168250
168250
179817
179817
179817
179817
179817
179817
179815
177651
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
182825
178061
178061
178061
177615
177615
177615
177615
177615
180497
180497
168608
168608
168608
168608
168418
181671
161644
177765
161644
177765
180987
179133
176371
176371
176371
176371

C:\Users\19178\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


26958
838
26958
261
261
261
5777
5777
5777
192385
192385
192385
192385
192385
192385
192385
192385
45880
4132
132532
2640
2640
2640
2640
26491
2779
2779
2779
2779
2779
2779
2779
2779
2779
1199
1199
6331
1224
1224
1224
3723
3723
1411
1411
1411
1411
5838
2429
1277
1277
1277
1277
1277
130073
97057
74789
74789
74789
4308
4308
4308
4308
4308
4308
4308
4308
1345
1345
8025
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
7
7
2476
41
41
1688
1688
170611
2112
187273
137857
82459
82459
82459
82459
82459
82459
82459
82459
82459
82459
74266
74266
74266
3201
3201
3201
3201
4296
4296
4296
4296
4296
4296
4296
1272
1272
1272
1272
1272
1272
1272
1272
1272
1272
8757
8757
8757
3037
7111
7111
7111
7111
6247
6247
6247
6247
78742
2520
2520
2520
2520
2520
2520
2520
2520
2520
2520
6849
6849
6849
6849
3066
3066
3066
3066
3066
8272
8272
74547
74547
74547
4282
7836
7836
7836
141454
52528
118778


1704
1704
1704
1633
1747
1747
1784
1784
1784
1784
1784
1784
1784
1729
1693
1693
1693
1693
1673
1673
1673
1733
1617
1617
1617
1617
1617
1617
1617
1617
1617
1614
1653
1730
1730
1730
1730
1780
4217
6667
4459
1608
1608
1569
1566
1552
1552
95654
1573
1676
1466
1701
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2333
2333
2333
2439
2439
2329
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2336
2336
2336
2336
2336
2336
2336
2272
2442
2442
1810
1810
2390
2321
2321
2321
2297
2297
2314
2337
2538
101062
3609
1912
1912
1907
2431
1917
1917
1917
1917
1881
2384
190219
4454
1883
3129
3129
2858
2858
2858
2858
2858
2858
2858
2858
3160
3160
3160
3118
2996
2996
2908
2908
3186
2997
2997
2997
3155
3155
3081
3081
3081
3163
3163
3163
3163
3185
3181
2677
7786
2824
3989
2683
3156
2587
3179
3114
2687
186343
2959
3157
2599
4021
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
4022
4022
4017
4017
3993
3993
3993
4036
4036
4034
4034
4034
4034
4034
3408
3408
3408
3408
3408
4014
401

166705
166705
161582
161582
161582
161582
165551
165551
165551
165551
165551
165551
161966
163959
178861
166486
166486
166486
155659
155659
166643
166643
166643
162578
162578
166461
166461
160848
152081
164179
164179
164179
164179
164179
164179
164179
164179
143367
164981
135143
135143
164019
167064
160874
160289
164917
169784
169788
169810
167870
156944
151113
151113
164949
165709
158972
135569
135536
166635
166635
165141
152782
143859
168750
165413
168752
168748
160718
169532
167036
169530
169534
162598
162598
160980
166528
166528
122922
133771
166788
168492
168492
168492
168492
181315
181315
181315
181315
181315
181315
168250
168250
168250
168250
179817
179817
179817
179817
179817
179817
179815
177651
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
182825
178061
178061
178061
177615
177615
177615
177615
177615
180497
180497
168608
168608
168608
168608
168418
181671
161644
177765
161644
177765
180987
179133
176371
176371
176371
176371
176371

C:\Users\19178\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y,preRatings_mode,preRatings_mean
160,1996,1997,69,COSTUME DESIGN,Ruth Myers,Emma,False,838,Emma (1996),Comedy|Drama|Romance,1996.0,5.0,4.076200
161,1996,1997,69,COSTUME DESIGN,Ruth Myers,Emma,False,26958,Emma (1996),Romance,1996.0,NaN,NaN
166,1996,1997,69,MUSIC (Original Musical or Comedy Score),Rachel Portman,Emma,True,838,Emma (1996),Comedy|Drama|Romance,1996.0,5.0,4.076200
167,1996,1997,69,MUSIC (Original Musical or Comedy Score),Rachel Portman,Emma,True,26958,Emma (1996),Romance,1996.0,NaN,NaN
225,1994,1995,67,ACTRESS IN A LEADING ROLE,Winona Ryder,Little Women,False,261,Little Women (1994),Drama,1994.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0,3.5,4.000000
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0,4.0,3.758564
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0,4.0,3.483051
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,3.5,3.464356


In [320]:
dfOscarMovies

,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y
2,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,25768,Sadie Thompson (1928),Drama,1928.0
7,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,True,87290,Two Arabian Knights (1927),Adventure|Comedy|Romance,1927.0
8,1927,1928,1,DIRECTING (Comedy Picture),Ted Wilde,Speedy,False,32551,Speedy (1928),Action|Comedy,1928.0
9,1927,1928,1,ENGINEERING EFFECTS,Roy Pomeroy,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
10,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0


In [329]:
# brief statistical analysis
import statsmodels.api as sm
dfOM_1970 = pd.read_csv('oscar_movies_preRat.csv')
dfOM_1970 = dfOM_1970.dropna()
numOfAwards = dfOM_1970[['movieId', 'winner']].groupby('movieId').count()
# mode of preRatings
preRat = dfOM_1970[['movieId', 'preRatings_mode']].groupby('movieId').mean()
model = sm.OLS(numOfAwards, preRat).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 winner   R-squared (uncentered):                   0.550
Model:                            OLS   Adj. R-squared (uncentered):              0.549
Method:                 Least Squares   F-statistic:                              743.0
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                   1.48e-107
Time:                        00:59:27   Log-Likelihood:                         -1406.2
No. Observations:                 610   AIC:                                      2814.
Df Residuals:                     609   BIC:                                      2819.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [330]:
# mean of preRatings
preRat = dfOM_1970[['movieId', 'preRatings_mean']].groupby('movieId').mean()
model = sm.OLS(numOfAwards, preRat).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 winner   R-squared (uncentered):                   0.551
Model:                            OLS   Adj. R-squared (uncentered):              0.551
Method:                 Least Squares   F-statistic:                              748.4
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                   4.39e-108
Time:                        00:59:29   Log-Likelihood:                         -1405.0
No. Observations:                 610   AIC:                                      2812.
Df Residuals:                     609   BIC:                                      2816.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [344]:
# add other control variables
X = dfOM_1970[['movieId', 'preRatings_mean', 'year_ceremony', 'category']].groupby('movieId').apply(lambda x: x.iloc[0]) # Index doesn't matter.
X = pd.get_dummies(X, columns=['category'])
model = sm.OLS(numOfAwards, X.iloc[:,1:]).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 winner   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     12.50
Date:                Fri, 18 Sep 2020   Prob (F-statistic):           1.58e-47
Time:                        01:18:18   Log-Likelihood:                -1265.9
No. Observations:                 610   AIC:                             2598.
Df Residuals:                     577   BIC:                             2743.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

In [347]:
def get_postRatings(df, rating_df, mode='mode'):
    # collect the ratings after March in this year
    d = datetime.datetime(max(1970,df['year_ceremony']), 3, 31, 0)
    print(d)
    timestamp1 = time.mktime(d.timetuple()) # local time
    if mode == 'mode':
        output = rating_df[(rating_df['timestamp']>timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mode().mean()
    else:
        output = rating_df[(rating_df['timestamp']>timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mean()
    print(df['movieId'])
    return output # return mode/mean
postRatingsMode = lambda x: get_postRatings(x, dfRatings)
postRatingsMean = lambda x: get_postRatings(x, dfRatings, 'mean')
dfOscarMovies['postRatings_mode'] = dfOscarMovies.apply(postRatingsMode, axis=1) 
dfOscarMovies['postRatings_mean'] = dfOscarMovies.apply(postRatingsMean, axis=1) 
dfOscarMovies.to_csv('oscar_movies_postRat.csv')
dfOscarMovies

1970-03-31 00:00:00
25768
1970-03-31 00:00:00
87290
1970-03-31 00:00:00
32551
1970-03-31 00:00:00
1925
1970-03-31 00:00:00
1925
1970-03-31 00:00:00
79874
1970-03-31 00:00:00
142644
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
85576
1970-03-31 00:00:00
85576
1970-03-31 00:00:00
122585
1970-03-31 00:00:00
122585
1970-03-31 00:00:00
124795
1970-03-31 00:00:00
124795
1970-03-31 00:00:00
124795
1970-03-31 00:00:00
73967
1970-03-31 00:00:00
95563
1970-03-31 00:00:00
83833
1970-03-31 00:00:00
78224
1970-03-31 00:00:00
78224
1970-03-31 00:00:00
72897
1970-03-31 00:00:00
123000
1970-03-31 00:00:00
83376
1970-03-31 00:00:00
83376
1970-03-31 00:00:00
83376
1970-03-31 00:00:00
88818
1970-03-31 00:00:00
88818
1970-03-31 00:00:00
8518
1970-03-31 00:00:00
8518
1970-03-31 00:00:00
124595
1970-03-31 00:00:00
87595
1970-03-31 00:00:00
122001
1970-03-31 00:00:00
1927
1970-03-31 00:00:00
1927
1970-03-31 00:00:00
1927
1970-03-31

944
1970-03-31 00:00:00
25842
1970-03-31 00:00:00
25842
1970-03-31 00:00:00
25833
1970-03-31 00:00:00
32349
1970-03-31 00:00:00
32349
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
119617
1970-03-31 00:00:00
134732
1970-03-31 00:00:00
134732
1970-03-31 00:00:00
134732
1970-03-31 00:00:00
60343
1970-03-31 00:00:00
181193
1970-03-31 00:00:00
86620
1970-03-31 00:00:00
112307
1970-03-31 00:00:00
163350
1970-03-31 00:00:00
163350
1970-03-31 00:00:00
151137
1970-03-31 00:00:00
81857
1970-03-31 00:00:00
81857
1970-03-31 00:00:00
81583
1970-03-31 00:00:00
594
1970-03-31 00:00:00
975
1970-03-31 00:00:00
47723
1970-03-31 00:00:00
1066
1970-03-31 00:00:00
162716
197

25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
2206
1970-03-31 00:00:00
2206
1970-03-31 00:00:00
2206
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
25868
1970-03-31 00:00:00
25868
1970-03-31 

9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
25907
1970-03-31 00:00:00
25907
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
8461
1970-03-31 00:00:00
8461
1970-03-31 00:00:00
123246
1970-03-31 00:00:00
123246
1970-03-31 00:00:00
85450
1970-03-31 00:00:00
85450
1970-03-31 00:00:00
85450
1970-03-31 00:00:00
86927
1970-03-31 00:00:00
124226
1970-03-31 00:00:00
124226
1970-03-31 00:00:00
124226
1970-03-31 00:00:00
147424
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
160804
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
171197
1970-03-31 00:00:00
171197
1970-03-31 0

62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
69773
1992-03-31 00:00:00
5838
1970-03-31 00:00:00
79925
1970-03-31 00:00:00
79925
1970-03-31 00:00:00
86337
1970-03-31 00:00:00
86337
1970-03-31 00:00:00
86337
1970-03-31 00:00:00
5440
1970-03-31 00:00:00
41762
1970-03-31 00:00:00
3606
1970-03-31 00:00:00
81859
1970-03-31 00:00:00
80290
1970-03-31 00:00:00
1026
1970-03-31 00:00:00
143211
1970-03-31 00:00:00
2430
1999-03-31 00:00:00
2429
1970-03-31 00:00:00
70204
1970-03-31 00:00:00
30808
1970-03-31 00:00:00
8491
1970-03-31 00:00:00
42546
1970-03-31 00:00:00
44100
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1970-03-31 00:00:00
3088
1970-03-31 00:00:00
3088
1970-03-31 00:00:00
934
1970-03-31 00:00:00
934
1970-03-31 00:00:00
934
1970-03-31 00:00:00
54121
1970-03-31 00:00:00
54121
1970-03-31 00:00:00
54121
1970-03-31 00:00:00
83343

...

41997
2006-03-31 00:00:00
41997
2006-03-31 00:00:00
41997
2006-03-31 00:00:00
41997
2006-03-31 00:00:00
38198
2006-03-31 00:00:00
33154
2006-03-31 00:00:00
34153
2006-03-31 00:00:00
49225
2006-03-31 00:00:00
157883
2006-03-31 00:00:00
158013
2006-03-31 00:00:00
186981
2006-03-31 00:00:00
41527
2006-03-31 00:00:00
44204
2006-03-31 00:00:00
115881
2006-03-31 00:00:00
95377
2006-03-31 00:00:00
53318
2006-03-31 00:00:00
55207
2006-03-31 00:00:00
126096
2006-03-31 00:00:00
62115
2006-03-31 00:00:00
34048
2006-03-31 00:00:00
64997
2006-03-31 00:00:00
34048
2006-03-31 00:00:00
64997
2006-03-31 00:00:00
34048
2006-03-31 00:00:00
64997
2006-03-31 00:00:00
41285
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
47423
2007-03-31 00:00:00
50274
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
48696
2007-03-31 00:00:00
48696
2

82767
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
78039
2011-03-31 00:00:00
79720
2011-03-31 00:00:00
76093
2011-03-31 00:00:00
76093
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
80906
2011-03-31 00:00:00
81158
2011-03-31 00:00:00
83293
2011-03-31 00:00:00
123415
2011-03-31 00:00:00
86781
2011-03-31 00:00:00
86432
2011-03-31 00:00:00
82527
2011-03-31 00:00:00
84570
2011-03-31 00:00:00
81847
2011-03-31 00:00:00
83803
2011-03-31 00:00:00
158326
2011-03-31 00:00:00
175379
2011-03-31 00:00:00
82461
2011-03-31 00:00:00
81782
2011-03-31 00:00:00
79293
2011-03-31 00:00:00
81512
201

116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
117881
2015-03-31 00:00:00
112556
2015-03-31 00:00:00
118900
2015-03-31 00:00:00
118900
2015-03-31 00:00:00
118997
2015-03-31 00:00:00
118997
2015-03-31 00:00:00
118997
2015-03-31 00:00:00
115617
2015-03-31 00:00:00
112175
2015-03-31 00:00:00
117444
2015-03-31 00:00:00
106452
2015-03-31 00:00:00
1

168250
2018-03-31 00:00:00
168250
2018-03-31 00:00:00
168250
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179815
2018-03-31 00:00:00
177651
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
182825
2018-03-31 00:00:00
178061
2018-03-31 00:00:00
178061
2018-03-31 00:00:00
178061
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
180497
2018-03-31 00:00:00
180497
2018-03-31 00:00:00
168608
2018-03-31 00:00:00
168608
2018-03-31 00:00:00
1

,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y,postRatings_mode,postRatings_mean
2,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,25768,Sadie Thompson (1928),Drama,1928.0,4.0,3.500000
7,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,True,87290,Two Arabian Knights (1927),Adventure|Comedy|Romance,1927.0,3.0,2.500000
8,1927,1928,1,DIRECTING (Comedy Picture),Ted Wilde,Speedy,False,32551,Speedy (1928),Action|Comedy,1928.0,3.5,3.521739
9,1927,1928,1,ENGINEERING EFFECTS,Roy Pomeroy,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0,4.0,3.701635
10,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0,4.0,3.701635
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0,NaN,NaN
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0,NaN,NaN
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0,NaN,NaN
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,NaN,NaN


In [5]:
import pandas as pd
import statsmodels.api as sm
dfOscarMovies = pd.read_csv('oscar_movies_postRat.csv')
dfOscarMovies = dfOscarMovies.dropna()
numOfAwards = dfOscarMovies[['movieId', 'winner']].groupby('movieId').count()
# mode of preRatings
postRat = dfOscarMovies[['movieId', 'postRatings_mode']].groupby('movieId').mean()
model = sm.OLS(postRat, numOfAwards).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:       postRatings_mode   R-squared (uncentered):                   0.550
Model:                            OLS   Adj. R-squared (uncentered):              0.549
Method:                 Least Squares   F-statistic:                              2706.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                        0.00
Time:                        10:25:27   Log-Likelihood:                         -5149.4
No. Observations:                2219   AIC:                                  1.030e+04
Df Residuals:                    2218   BIC:                                  1.031e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [16]:
def get_preRatings(df, rating_df, mode='mode'):
    # collect the ratings before Feb in next year (oscar ceremony year)
    d = datetime.datetime(int(df['year_film']+1), 2, 1, 0)
    timestamp1 = time.mktime(d.timetuple()) # local time
    if mode == 'mode':
        output = rating_df[(rating_df['timestamp']<timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mode().mean()
    else:
        output = rating_df[(rating_df['timestamp']<timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mean()
    print(df['movieId'])
    return output # return mode/mean
# preRatingsMode = lambda x: get_preRatings(x, dfRatings)
preRatingsMean = lambda x: get_preRatings(x, dfRatings, 'mean')
dfMovies= dfMovies[dfMovies['year_film']>=1970] # for get the preRatings, use the dataset after 1970
# dfMovies['preRatings_mode'] = dfMovies.apply(preRatingsMode, axis=1) 
dfMovies['preRatings'] = dfMovies.apply(preRatingsMean, axis=1) 
dfMovies.to_csv('movies_preRat.csv')
dfMovies

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281

2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2177
2178
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2286
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2364
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411


4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4179
4180
4181
4182
4183
4185
4193
4195
4197
4198
4199
4200
4201
4202
4203
4204
4205
4206
4207
4208
4209
4210
4211
4212
4213
4214
4215
4216
4217
4218
4219
4220
4221
4222
4223
4224
4225
4226
4227
4228
4229
4230
4231
4232
4233
4234
4235
4236
4237
4238
4239
4240
4241
4242
4243
4244
4245
4246
4247
4248
4249
4250
4251
4252
4253
4254
4255
4256
4257
4258
4259
4260
4261
4262
4265
4266
4267
4268
4269
4270
4271
4272
4273
4275
4276
4279
4280
4283
4285
4286
4288
4289
4290
4291
4292
4293
4295
4296
4297
4299
4300
4301
4302
4303
4304
4305
4306
4307
4308
4309
4310
4311
4312
4313
4314
4315
4316
4317
4318
4320
4321
4322
4326
4331
4332
4333
4334
4335
4336
4340
4341
4342
4343
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4361
4362
4363
4364
4365
4366
4367
4368
4369
4370
4371
4372
4373
4374
4375
4376
4377
4378
4379
4380
4381
4382
4383
4384
4385
4386
4387
4388
4389
4390
4391
4392
4393
4396
4397
4398
4407
4408
4409
4410
4412
4415
4417
4418
4421


6067
6070
6074
6075
6076
6077
6078
6079
6080
6081
6082
6083
6084
6085
6086
6087
6088
6089
6090
6091
6092
6093
6094
6095
6096
6097
6098
6099
6100
6101
6102
6103
6104
6105
6106
6107
6108
6109
6110
6111
6112
6113
6114
6115
6116
6117
6118
6119
6120
6121
6122
6123
6124
6125
6126
6127
6128
6129
6130
6131
6132
6133
6134
6135
6136
6137
6138
6139
6140
6141
6142
6143
6144
6145
6146
6147
6148
6149
6150
6151
6152
6153
6154
6155
6156
6157
6158
6159
6160
6161
6162
6163
6164
6165
6166
6168
6169
6170
6171
6173
6174
6175
6176
6177
6179
6180
6184
6185
6186
6187
6188
6189
6190
6191
6192
6193
6194
6195
6196
6197
6198
6199
6200
6201
6202
6203
6204
6205
6206
6207
6209
6210
6211
6212
6213
6214
6215
6216
6217
6218
6219
6220
6221
6222
6223
6224
6226
6229
6230
6233
6234
6235
6238
6240
6241
6242
6243
6244
6246
6248
6249
6250
6251
6252
6253
6256
6261
6262
6263
6264
6265
6266
6267
6268
6269
6270
6271
6272
6274
6275
6277
6278
6279
6280
6281
6282
6283
6284
6285
6286
6287
6288
6289
6290
6291
6292
6293
6294
6295
6296


8907
8908
8909
8910
8911
8912
8913
8914
8915
8916
8917
8918
8919
8922
8923
8924
8925
8926
8927
8929
8930
8931
8932
8933
8934
8935
8936
8937
8938
8939
8940
8941
8942
8943
8944
8945
8946
8947
8948
8949
8950
8951
8952
8953
8954
8955
8956
8957
8958
8959
8960
8961
8962
8963
8964
8965
8966
8967
8968
8969
8970
8971
8972
8973
8974
8975
8977
8978
8979
8980
8981
8982
8983
8984
8985
8987
8996
8998
8999
9000
9004
9005
9009
9010
9013
9015
9016
9017
9018
9019
26203
26214
26216
26219
26220
26221
26222
26223
26225
26226
26227
26228
26229
26230
26231
26232
26235
26236
26237
26240
26241
26242
26243
26245
26246
26247
26248
26249
26251
26252
26253
26254
26255
26256
26257
26258
26259
26263
26264
26265
26266
26267
26268
26269
26270
26271
26274
26277
26280
26282
26283
26285
26287
26288
26289
26290
26291
26294
26295
26297
26299
26301
26302
26303
26304
26306
26307
26308
26309
26312
26313
26314
26315
26316
26317
26318
26320
26321
26322
26323
26324
26325
26326
26327
26333
26334
26336
26337
26338
26339
26340
2634

...

182525
182527
182533
182547
182549
182551
182553
182555
182557
182559
182561
182563
182565
182567
182569
182581
182585
182587
182591
182593
182595
182597
182599
182601
182605
182607
182609
182613
182615
182617
182619
182623
182625
182627
182629
182631
182633
182635
182637
182639
182641
182643
182645
182647
182649
182651
182653
182655
182657
182659
182661
182663
182665
182667
182669
182671
182673
182675
182679
182681
182683
182685
182689
182691
182693
182695
182697
182699
182701
182703
182705
182707
182709
182711
182713
182715
182717
182719
182721
182725
182727
182729
182731
182733
182735
182737
182739
182741
182743
182745
182747
182749
182751
182755
182757
182759
182761
182763
182767
182769
182771
182773
182775
182777
182779
182781
182783
182785
182787
182789
182791
182793
182797
182803
182805
182807
182811
182813
182815
182817
182819
182821
182823
182825
182827
182829
182831
182833
182835
182837
182841
182843
182847
182849
182851
182853
182855
182857
182859
182861
182863
182865
182867

185311
185313
185315
185317
185319
185321
185323
185325
185329
185331
185335
185337
185339
185341
185343
185345
185347
185349
185353
185355
185359
185361
185363
185365
185373
185375
185377
185379
185381
185385
185387
185389
185391
185393
185395
185397
185399
185401
185403
185405
185407
185409
185411
185413
185415
185417
185419
185421
185423
185425
185427
185429
185431
185433
185435
185437
185439
185441
185443
185445
185449
185453
185459
185461
185463
185467
185469
185471
185473
185475
185479
185481
185483
185485
185489
185493
185497
185503
185511
185513
185515
185517
185519
185521
185523
185525
185527
185531
185533
185537
185539
185541
185543
185545
185547
185549
185551
185553
185555
185561
185563
185569
185571
185573
185575
185577
185579
185581
185583
185585
185587
185591
185593
185595
185597
185601
185603
185605
185609
185611
185613
185619
185621
185623
185625
185627
185629
185631
185633
185635
185637
185639
185641
185643
185645
185647
185649
185651
185653
185655
185657
185659
185661

188187
188189
188191
188193
188195
188197
188199
188205
188207
188209
188211
188213
188215
188217
188221
188223
188225
188227
188229
188231
188233
188235
188239
188241
188249
188251
188255
188259
188261
188265
188267
188269
188271
188273
188275
188279
188283
188285
188287
188291
188293
188295
188297
188299
188301
188303
188305
188307
188309
188311
188313
188315
188317
188331
188333
188335
188337
188339
188341
188343
188349
188351
188353
188355
188357
188359
188361
188363
188365
188371
188373
188375
188377
188379
188381
188383
188385
188387
188389
188391
188393
188397
188399
188401
188403
188405
188409
188411
188413
188415
188417
188419
188421
188427
188429
188431
188433
188435
188439
188441
188445
188447
188449
188453
188455
188457
188459
188465
188477
188491
188493
188495
188497
188501
188503
188505
188507
188511
188513
188515
188519
188521
188523
188525
188527
188529
188531
188533
188535
188537
188539
188541
188543
188545
188547
188549
188551
188553
188555
188557
188559
188561
188565

191407
191409
191411
191413
191415
191417
191419
191421
191423
191425
191427
191429
191431
191433
191437
191439
191441
191443
191445
191447
191449
191451
191453
191455
191457
191459
191465
191469
191471
191473
191475
191477
191479
191483
191485
191487
191489
191491
191495
191499
191501
191503
191505
191511
191513
191515
191517
191561
191573
191579
191601
191605
191635
191637
191643
191647
191651
191653
191655
191657
191659
191661
191663
191665
191667
191669
191673
191675
191677
191679
191681
191683
191685
191687
191689
191691
191693
191695
191699
191701
191703
191707
191709
191711
191713
191715
191717
191719
191721
191723
191725
191727
191729
191731
191733
191735
191737
191739
191741
191743
191745
191747
191749
191751
191753
191755
191757
191759
191761
191763
191765
191767
191769
191771
191773
191775
191777
191779
191781
191783
191785
191787
191789
191791
191793
191795
191797
191799
191803
191805
191807
191809
191811
191813
191819
191821
191823
191829
191831
191833
191835
191837
191841

,movieId,title,genres,film,year_film,preRatings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995.0,4.00
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995.0,4.00
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995.0,5.00
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995.0,NaN
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995.0,4.00
...,...,...,...,...,...,...
58091,193872,Room Laundering (2018),Drama,Room Laundering,2018.0,4.00
58094,193878,Les tribulations d'une caissière (2011),Comedy,Les tribulations d'une caissière,2011.0,NaN
58095,193880,Her Name Was Mumu (2016),Drama,Her Name Was Mumu,2016.0,NaN
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi,Flora,2017.0,NaN
